# Part 1

## Chapter 1.4

### Section 1.4.1

In [ ]:
import pandas as pd
from scipy import stats
import joblib
import numpy as np
from collections import namedtuple

In [ ]:
df = pd.read_csv('http://www.minethatdata.com/Kevin_Hillstrom_MineThatData_E-MailAnalytics_DataMiningChallenge_2008.03.20.csv')

In [ ]:
df.head()

In [ ]:
# Implement (4)


In [ ]:
df_male.head()

In [ ]:
# TEST (4)
assert type(df_male) is pd.DataFrame  # 女性向けメールが配信されなかったグループのみ含むdf
assert "Womens E-Mail" not in df_male["segment"]  # 女性向けメールが配信されていない
assert set(df_male[df_male["treatment"] == 1]["segment"]) == set(["Mens E-Mail"])  # treatmentには男性向けメールが配信されたグループのみ
assert set(df_male.keys()) == set(df.keys()).union(set(["treatment"]))

### Section 1.4.2

In [ ]:
# Implement (5)


In [ ]:
df_aggregated

In [ ]:
# TEST (5)
assert type(df_aggregated) is pd.DataFrame  # 集計されたdf
assert set(["conversion_rate", "spend_mean", "count"]).issubset(df_aggregated.keys())  # 指定のカラムを持つ
assert (df_aggregated.sort_values("treatment")["conversion_rate"].values - np.array([0.005726, 0.01253])).sum() < 1e-4
assert (df_aggregated.sort_values("treatment")["spend_mean"].values - np.array([0.652789, 1.422617])).sum() < 1e-4
assert (df_aggregated.sort_values("treatment")["count"].values - np.array([21306, 21307])).sum() < 1e-4

In [ ]:
# Implement (6)


In [ ]:
pvalue

In [ ]:
# TEST (6)
assert type(pvalue) is float  # result of stats.ttest_ind
assert pvalue < 0.05  # 有意

### Section 1.4.3

In [ ]:
# Implement (7) - No tests. これ以降の乱数の影響をなくすため
bias = (df_male.history > 300) | (df_male.recency < 6) | (df_male.channel=='Multichannel')
df_biased = pd.concat([
    df_male[(df_male["treatment"] == 0) & bias].sample(frac=0.5, random_state=1),
    df_male[(df_male["treatment"] == 0) & ~bias],
    df_male[(df_male["treatment"] == 1) & bias],
    df_male[(df_male["treatment"] == 1) & ~bias].sample(frac=0.5, random_state=1),
], ignore_index=True)

In [ ]:
df_biased.head()

In [ ]:
# Implement (8)


In [ ]:
df_aggregated_biased

In [ ]:
# TEST (8)
assert type(df_aggregated_biased) is pd.DataFrame
assert (
    df_aggregated_biased.sort_values("treatment")["conversion_rate"][0]
    < df_aggregated.sort_values("treatment")["conversion_rate"][0]
), "conversion_rate drops in controll due to bias"
assert (
    df_aggregated_biased.sort_values("treatment")["conversion_rate"][1]
    > df_aggregated.sort_values("treatment")["conversion_rate"][1]
), "conversion_rate increase in treatment due to bias"
assert (
    df_aggregated_biased.sort_values("treatment")["spend_mean"][1]
    > df_aggregated.sort_values("treatment")["spend_mean"][1]
), "spend_mean increase in treatment due to bias"

In [ ]:
# Implement (9)


In [ ]:
pvalue_biased

In [ ]:
# TEST (9)
assert type(pvalue_biased) is float
assert pvalue_biased < 0.05  # バイアスがあるが有意という結果が出るはず"
assert pvalue_biased < ttest_result.pvalue  # バイアスのせいでp値が小さくなるはず

# Part 2

## Chapter 2.1

### Section 2.1.1

In [ ]:
X_biased = df_biased[["treatment", "history"]]
y_biased = df_biased["spend"]

In [ ]:
# Implement (6)


In [ ]:
result_biased.summary()

In [ ]:
# TEST (6)
assert type(beta_treatment_biased) is float
assert type(pvalue_treatment_biased) is float
assert type(beta_history_biased) is float
assert type(pvalue_history_biased) is float
assert abs(beta_treatment_biased - 0.9026) < 1e-2
assert pvalue_treatment_biased < 0.05  # 有意
assert abs(beta_history_biased - 0.0010) < 1e-2

## Chapter 2.2

### Section 2.2.1

In [ ]:
X = df_male[["treatment"]]
y = df_male["spend"]

X_biased = df_biased[["treatment"]]
y_biased = df_biased["spend"]

X_biased_mreg = df_biased[["treatment", "recency", "channel", "history"]].copy()
X_biased_mreg["channel"] = (X_biased_mreg["channel"] == "Web").astype(int)  # 2種しかないカテゴリカル変数なのでバイナリに変換
y_biased_mreg = df_biased["spend"]

In [ ]:
# Implement (7)


In [ ]:
beta_treatment, beta_treatment_biased, beta_treatment_biased_mreg

In [ ]:
# TEST (7)
assert type(beta_treatment) is float
assert type(pvalue_treatment) is float
assert type(beta_treatment_biased) is float
assert type(pvalue_treatment_biased) is float
assert type(beta_treatment_biased_mreg) is float
assert type(pvalue_treatment_biased_mreg) is float

assert abs(beta_treatment - 0.770) < 1e-2
gain_from_df_aggregated = df_aggregated["spend_mean"][1] - df_aggregated["spend_mean"][0]
assert abs(beta_treatment - gain_from_df_aggregated) < 1e-8  # 1章の結果と同じになるはず
assert pvalue_treatment < 0.05  # 有意
assert abs(beta_treatment_biased - 0.979) < 1e-2  # バイアスによりbeta_treatmentより大きくなる
assert pvalue_treatment_biased < 0.05  # 有意
assert beta_treatment_biased_mreg < beta_treatment_biased  # 共変量を加えてバイアスの影響が小さくなる

### Section 2.2.3

In [ ]:
X_biased_omit = df_biased[["treatment", "recency", "channel"]].copy()  # historyを抜いたdf
X_biased_omit["channel"] = (X_biased_omit["channel"] == "Web").astype(int)  # 2種しかないカテゴリカル変数なのでバイナリに変換
y_biased_omit = df_biased["spend"]

X_biased = df_biased[["treatment", "recency", "channel", "history"]].copy()
X_biased["channel"] = (X_biased["channel"] == "Web").astype(int)  # 2種しかないカテゴリカル変数なのでバイナリに変換
y_biased = df_biased["spend"]

X_biased_history = df_biased[["treatment", "recency", "channel"]].copy()
X_biased_history["channel"] = (X_biased_history["channel"] == "Web").astype(int)  # 2種しかないカテゴリカル変数なのでバイナリに変換
y_biased_history = df_biased["history"]

In [ ]:
# Implement (9)


In [ ]:
beta_history * gamma_treatment, (alpha_treatment - beta_treatment)

In [ ]:
# TEST (9)
assert type(alpha_treatment) is float  # モデルAにおけるtreatmentの係数
assert type(beta_treatment) is float  # モデルBにおけるtreatmentの係数
assert type(beta_history) is float  # モデルBにおけるhistoryの係数
assert type(gamma_treatment) is float  # モデルCにおけるtreatmentの係数

assert abs(beta_history * gamma_treatment - (alpha_treatment - beta_treatment)) < 1e-8  # 知られている式

### Section 2.2.7

In [ ]:
X_biased = df_biased[["treatment", "recency", "channel", "history"]].copy()
X_biased["channel"] = (X_biased["channel"] == "Web").astype(int)  # 2種しかないカテゴリカル変数なのでバイナリに変換
y_biased = df_biased["spend"]

X_biased_visit = df_biased[["treatment", "recency", "channel", "history", "visit"]].copy()
X_biased_visit["channel"] = (X_biased_visit["channel"] == "Web").astype(int)  # 2種しかないカテゴリカル変数なのでバイナリに変換
y_biased_visit = df_biased["spend"]

In [ ]:
# Implement (10)


In [ ]:
cor_spend_visit, pvalue_spend_visit

In [ ]:
beta_treatment_biased, beta_treatment_biased_visit

In [ ]:
# TEST (10)
assert type(cor_spend_visit) is float  # spendとvisitの相関係数
assert type(pvalue_spend_visit) is float  # spendとvisitの相関のp値
assert type(beta_treatment_biased) is float  # treatmentの効果
assert type(beta_treatment_biased_visit) is float  # treatmentの効果(visitが変数に入る場合)

assert abs(cor_spend_visit - 0.144) < 5e-2
assert pvalue_spend_visit < 0.05  # 有意

assert beta_treatment_biased > beta_treatment_biased_visit  # visitのせいでtreatmentの介入効果が低く見える

In [ ]:
cor_spend_visit, pvalue_spend_visit